
# Data Generation

To evaluate how we can create features from point in time data , we'll want to create two datasets that are representative of Health Care data. Specifically:
 - `main.default.patient_lab` - This will include generated patient lab results
 - `main.default.patient_lab` - 



## `patient_lab`

| Lab | lab_types | Description |
| --- | -------- | ----------- |
| Urinalysis (UA) | `ua_ph`</br>`ua_protein`</br>`ua_glucose`</br>`ua_ketones`| Analyzes urine components; pH, protein, glucose, ketones |
| Blood Urea Nitrogen (BUN) | `bun` | Measures kidney function. |
| Creatinine | `creatinine` | Waste product indicating kidney function. |
| Glomerular Filtration Rate (GFR) | `gfr` | Estimates kidney function efficiency. |

First, we'll create a table that has a typical format which is that each lab results has it's own record. To demo functionality, we'll generate some sample data. To be able to later test the scalability of the solution, you can adjust the configurations in the generation code below.

In [0]:
%sql

CREATE TABLE IF NOT EXISTS main.default.patient_lab (
    patient_id LONG    COMMENT "Unique identifier for a patient",
    event_ts TIMESTAMP COMMENT "Timestamp of when the lab was sample was taken",
    lab_type STRING    COMMENT "The specific lab metric recorded",
    lab_value STRING   COMMENT "The numeric reulsts of the lab_type")

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
from pyspark.sql.functions import col

# Function to generate random dates within a year, ensuring at least one week apart
def generate_random_dates(year, num_dates):
    start_date = datetime(year, 1, 1)
    end_date = datetime(year, 12, 31)
    dates = []
    while len(dates) < num_dates:
        random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
        if all(abs((random_date - d).days) >= 7 for d in dates):
            dates.append(random_date)
    return sorted(dates)

# Generate sample data
data = []
patient_ids = range(1, 11)
years = [2023, 2024]
lab_types = ['ua_ph', 'ua_protein', 'ua_glucose', 'ua_ketones']

for patient_id in patient_ids:
    for year in years:
        dates = generate_random_dates(year, 5)
        for date in dates:
            lab_data = {
                'patient_id': patient_id,
                'date': date,
                'ua_ph': round(random.uniform(4.5, 8.0), 1),  # Normal range for urine pH
                'ua_protein': random.choice(['negative', 'trace', '1+', '2+', '3+']),  # Normal is 'negative' or 'trace'
                'ua_glucose': random.choice(['negative', 'trace', '1+', '2+', '3+']),  # Normal is 'negative'
                'ua_ketones': random.choice(['negative', 'trace', '1+', '2+', '3+'])   # Normal is 'negative'
            }
            data.append(lab_data)

# Define lab_data Spark DataFrame
lab_data = spark.createDataFrame(pd.DataFrame(data)) \
                .withColumnRenamed('date', 'event_ts') \
                .withColumn("ua_ph", col("ua_ph").cast("bigint")) \
                .withColumn("ua_ph", col("ua_ph").cast("string")) \
                .selectExpr("patient_id",
                            "event_ts",
                            """stack(4, 'ua_ph', ua_ph,
                                        'ua_protein', ua_protein,
                                        'ua_glucose', ua_glucose,
                                        'ua_ketones', ua_ketones) as (lab_type, lab_value)""")                             

# Write lab_data to Delta Lake
lab_data.write.mode("overwrite").saveAsTable("main.default.patient_lab")

# Display the DataFrame
# display(lab_data)

In [0]:
%sql
SELECT * FROM main.default.patient_lab limit 8;


## `patient_event`

The records for patient event will include at a minimum the unique identifier for the patient, `patient_id`, and the time of the event, `event_ts`.

**NOTE**: While both the labs table and the patient_event table will both have the field `event_ts`, these fields will likely not matching and have different meanings. Specifically:
 - labs_tsdf `event_ts`: The time that lan sample was taken and evaluated. 
 - patient_event `event_ts`: The event time that would trigger an evaluation. This could be any event time when a model may be run requiring augmented features. For example, a patient event where a model could be run would be at patient checkout to evaluate probability of readmittance.


In [0]:
%sql

CREATE TABLE IF NOT EXISTS main.default.patient_event (
  patient_id STRING COMMENT "Unique identifier for a patient",
  last_name STRING COMMENT "Patient's last name",
  first_name STRING COMMENT "Patient's first name",
  event_ts TIMESTAMP COMMENT "Timestamp of the event");

INSERT OVERWRITE main.default.patient_event
    VALUES ('3', 'Smith', 'John', '2024-01-18 12:00:00'),
           ('5', 'Mary',  'Mac',  '2024-02-11 12:00:00');

SELECT * FROM main.default.patient_event;